<a href="https://colab.research.google.com/github/amanotk/numerical-geophysics/blob/main/notebook/BurgesEquation.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
</a>

# Burgers方程式

以下のBurgers方程式の数値解を差分法によって求める．
$$
\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} =
\alpha \frac{\partial^2 u}{\partial x^2}
$$

In [ ]:
# 準備
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

In [ ]:
# フォントサイズ
plt.rcParams["font.size"] = 14

## 非粘性Burgers方程式の衝撃波解および膨張波解

$\alpha = 0$の非粘性Burgers方程式を考える．
計算領域は$-1 \leq x \leq +1$で境界条件は周期境界条件，初期条件は
$$
u(x) =
\begin{cases}
& 0 \quad -1 < x < -\frac{1}{3} \\
& 1 \quad -\frac{1}{3} < x < +\frac{1}{3} \\
& 0 \quad +\frac{1}{3} < x < 1 \\
\end{cases}
$$
とする．$x = +1/3$の不連続は衝撃波，$x = -1/3$の不連続は膨張波を形成する．

膨張波と衝撃波が交わるまで($t < 4/3$)の解析解は
$$
u(x, t) =
\begin{cases}
& 0 & -1 < x < x_1 \\
& (x-x_1)/(x_2 - x_1) & x_1 < x < x_2 \\
& 1 & x_2 < x < x_3 \\
& 0 & x_3 < x < +1
\end{cases}
$$
で与えられる．ここで$x_1 = -1/3$, $x_2 = t - 1/3$, $x_3 = t/2 + 1/3$である．

### 1次精度風上差分スキーム
ここでは（数値振動が生じない限りは）常に$u > 0$である．これを仮定すると以下の3通りのスキームが考えられる．
$$
\begin{align}
\frac{u^{n+1}_{i} - u^{n}_{i}}{\Delta t} &+
\frac{u^{n}_{i} + u^{n}_{i-1}}{2} \frac{u^{n}_{i} - u^{n}_{i-1}}{\Delta x}
= 0 \tag{A} \\
\frac{u^{n+1}_{i} - u^{n}_{i}}{\Delta t} &+
u^{n}_{i} \frac{u^{n}_{i} - u^{n}_{i-1}}{\Delta x}
= 0 \tag{B} \\
\frac{u^{n+1}_{i} - u^{n}_{i}}{\Delta t} &+
u^{n}_{i-1} \frac{u^{n}_{i} - u^{n}_{i-1}}{\Delta x}
= 0 \tag{C} \\
\end{align}
$$


In [ ]:
def set_initial(Nx):
  dx = 2.0/Nx
  xx = dx*np.arange(Nx+2) - (1 + 0.5*dx)
  uu = np.where(np.abs(xx) < 1/3, 1.0, 0.0)
  return xx, dx, uu

def analytic_solution(t, x):
  x1 = - 1/3
  x2 = t - 1/3
  x3 = t/2 + 1/3
  i1 = np.argwhere(np.logical_and(x1 < x, x < x2))
  i2 = np.argwhere(np.logical_and(x2 < x, x < x3))
  uu = np.zeros_like(x)
  uu[i1] = (x[i1] - x1)/(x2 - x1)
  uu[i2] = 1
  return uu

def push_upwind(u, dt, dx, step, scheme):
  Nx = np.size(u) - 2
  ix = np.arange(1, Nx+1, dtype=np.int32)
  if scheme == 'A':
    for n in range(step):
      # 更新
      u[ix] = u[ix] - 0.5*(u[ix]+u[ix-1])*dt/dx*(u[ix] - u[ix-1])
      # 境界条件
      u[   0] = u[Nx]
      u[Nx+1] = u[ 1]
  elif scheme == 'B':
    for n in range(step):
      # 更新
      u[ix] = u[ix] - u[ix]*dt/dx*(u[ix] - u[ix-1])
      # 境界条件
      u[   0] = u[Nx]
      u[Nx+1] = u[ 1]
  elif scheme == 'C':
    for n in range(step):
      # 更新
      u[ix] = u[ix] - u[ix-1]*dt/dx*(u[ix] - u[ix-1])
      # 境界条件
      u[   0] = u[Nx]
      u[Nx+1] = u[ 1]
  else:
    raise ValueError('No such scheme')

# パラメータ
Nx = 200
T  = 0.5
nu = 0.5
ix = np.arange(1, Nx+1)

# 初期条件
xx, dx, uu = set_initial(Nx)
dt = nu*dx
step = int(T/dt)

# プロット
fig, axs = plt.subplots(figsize=(10, 6))

# A
ua = uu.copy()
push_upwind(ua, dt, dx, step, 'A')
plt.plot(xx[ix], ua[ix], label='A')

# B
ub = uu.copy()
push_upwind(ub, dt, dx, step, 'B')
plt.plot(xx[ix], ub[ix], label='B')

# C
uc = uu.copy()
push_upwind(uc, dt, dx, step, 'C')
plt.plot(xx[ix], uc[ix], label='C')

# 解析解
uu = analytic_solution(T, xx)
plt.plot(xx[ix], uu[ix], 'k-', label='Analytic')

plt.xlim(-1.0, 1.0)
plt.ylim(-0.1, +1.2)
plt.xlabel('x')
plt.ylabel('u')
plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
plt.suptitle('t = {:4.2f}'.format(T))

## 滑らかな初期条件からの衝撃波の形成

計算領域を$0 \leq x \leq 1$とし，初期条件を
$$
u(x) = \sin \left( 2\pi x \right)
$$
とする．このような滑らかな初期条件からスタートしても有限の時間で解の急峻化が起こり，衝撃波が形成する．ここでは保存形の数値計算スキームを考え，数値流速$\hat{f}_{i+1/2}$の求め方としては以下の以下の2種類の方法を考えよう．

### 1次精度風上差分スキーム
$$
\hat{f}_{i+1/2} = \frac{f_{i} + f_{i+1}}{2} -
\frac{|a_{i+1/2}|}{2} (u_{i+1} - u_{i}) -
\alpha \frac{u_{i+1} - u_{i}}{\Delta x}
$$
ただし，$f_{i} = (u_{i})^2 / 2$である．ここで
$$
a_{i+1/2} = \left( \frac{\partial f}{\partial u} \right)_{i+1/2} =
\frac{f_{i+1} - f_{i}}{u_{i+1} - u_{i}} =
\frac{u_{i} + u_{i+1}}{2}
$$
はセル境界での位相速度である．
（実はこれは線形化された近似Riemann解法とみなすことができる．）


### 2段階Lax-Wendroffスキーム
$$
\begin{aligned}
& u_{i+1/2} = \frac{u_{i} + u_{i+1}}{2} - \frac{\Delta t}{2 \Delta x}
\left( f_{i+1} - f_{i} \right)
\\
& \hat{f}_{i+1/2} = \frac{(u_{i+1/2})^2}{2} -
\kappa_{i+1/2} \frac{\Delta x}{\Delta t} (u_{i+1} - u_{i}) -
\alpha \frac{u_{i+1} - u_{i}}{\Delta x}
\end{aligned}
$$
ここで
$$
\kappa_{i+1/2} = \varepsilon |u_{i+1} - u_{i}|
$$
である．

In [ ]:
def set_initial(Nx):
  dx = 1.0/Nx
  xx = dx*np.arange(Nx+2) - 0.5*dx
  uu = np.sin(2*np.pi*xx)
  return xx, dx, uu

def push_upwind(u, dt, dx, alpha, step):
  Nx = np.size(u) - 2
  ix = np.arange(1, Nx+1, dtype=np.int32)
  hx = np.arange(0, Nx+1, dtype=np.int32) # i+1/2
  fx = np.zeros_like(u)
  for n in range(step):
    # 更新
    fx[hx] = 0.25*(u[hx+1]**2 + u[hx]**2) \
      - 0.25*np.abs(u[hx+1]+u[hx])*(u[hx+1] - u[hx]) \
      - alpha*(u[hx+1] - u[hx])/dx
    u[ix] = u[ix] - dt/dx*(fx[ix] - fx[ix-1])
    # 境界条件
    u[   0] = u[Nx]
    u[Nx+1] = u[ 1]

def push_lw2(u, dt, dx, alpha, step, epsilon):
  Nx = np.size(u) - 2
  ix = np.arange(1, Nx+1, dtype=np.int32)
  hx = np.arange(0, Nx+1, dtype=np.int32) # i+1/2
  fx = np.zeros_like(u)
  for n in range(step):
    # 人工粘性の係数を決定
    kappa = epsilon*np.abs(u[hx+1] - u[hx])
    # 2段階Lax-Wendroffスキーム
    uh = 0.5*(u[hx] + u[hx+1]) - 0.25*dt/dx*(u[hx+1]**2 - u[hx]**2)
    fx[hx] = 0.5*uh**2 - (alpha + kappa*dx*dx/dt)*(u[hx+1] - u[hx])/dx
    u[ix] = u[ix] - dt/dx*(fx[ix] - fx[ix-1])
    # 境界条件
    u[   0] = u[Nx]
    u[Nx+1] = u[ 1]

# パラメータ
Nx = 200
nu = 0.2
ix = np.arange(1, Nx+1)
alpha = 1.0e-3
epsilon = 0.1

# 初期条件
xx, dx, uu = set_initial(Nx)
dt = nu*dx

print('alpha*dt/dx**2 = {:10.5e}'.format(alpha*dt/dx**2))

# プロット
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

axs[0].set_title('First-order Upwind')
axs[1].set_title('Two-step Lax-Wendroff')
u0 = uu.copy()
u1 = uu.copy()

step = 100
for n in range(5):
  axs[0].plot(xx[ix], u0[ix], label='t = {:5.3f}'.format(n*step*dt))
  axs[1].plot(xx[ix], u1[ix], label='t = {:5.3f}'.format(n*step*dt))
  push_upwind(u0, dt, dx, alpha, step)
  push_lw2(u1, dt, dx, alpha, step, epsilon)

for ax in axs:
  ax.set_xlim(0.0, 1.0)
  ax.set_ylabel('u')
  ax.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))

axs[0].set_xlabel('')
axs[1].set_xlabel('x')